In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, random_split
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torchvision.models import resnet18
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from collections import Counter
from tqdm import tqdm
import torchvision
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [28]:
def load_data():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    return train_set, test_set

def create_model():
    model = resnet18(pretrained=True)
    num_features = model.fc.in_features
    model.fc = torch.nn.Linear(num_features, 10)  # CIFAR-10 has 10 classes
    model = model.to(device)  # Move model to GPU
    return model

def train_shard(model, data_loader, epochs):
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    model.train()
    for epoch in tqdm(range(epochs), desc=f"Training for {epochs} epochs"):
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    return model

def sisa_training(dataset, S, R, epochs_list):
    shard_size = len(dataset) // S
    shards = random_split(dataset, [shard_size] * S)  # Creating shards
    models = []

    for shard in shards:
        slice_size = len(shard) // R
        slices = random_split(shard, [slice_size] * R)  # Creating slices
        model = create_model()
        for i, slice in enumerate(slices):
            data_loader = DataLoader(slice, batch_size=32, shuffle=True)
            model = train_shard(model, data_loader, epochs_list[i % len(epochs_list)])
        models.append(model)
    return models

def aggregate_majority_vote(models, dataloader):
    for model in models:
        model.eval()

    all_preds = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)  # Move data to the GPU
            outputs = [model(inputs) for model in models]
            preds = [torch.argmax(output, dim=1) for output in outputs]
            # Majority vote
            preds = torch.stack(preds, dim=1)
            majority_preds = [Counter(preds[i].tolist()).most_common(1)[0][0] for i in range(preds.size(0))]
            all_preds.extend(majority_preds)

    return torch.tensor(all_preds)

def aggregate_averaging_vectors(models, dataloader):
    for model in models:
        model.eval()

    all_outputs = []
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)  # Move data to the GPU
            outputs = [model(inputs) for model in models]
            avg_output = torch.mean(torch.stack(outputs), dim=0)
            all_outputs.append(avg_output)

    return torch.cat(all_outputs)

def evaluate_model(models, test_loader, aggregation_method):
    if aggregation_method == "majority_vote":
        preds = aggregate_majority_vote(models, test_loader)
        labels = torch.cat([labels for _, labels in test_loader]).to(device)  # Move labels to the GPU
    elif aggregation_method == "averaging_vectors":
        aggregated_outputs = aggregate_averaging_vectors(models, test_loader)
        _, preds = torch.max(aggregated_outputs, 1)
        labels = torch.cat([labels for _, labels in test_loader]).to(device)  # Move labels to the GPU

    f1 = f1_score(labels.cpu(), preds.cpu(), average='weighted')
    accuracy = accuracy_score(labels.cpu(), preds.cpu())
    precision = precision_score(labels.cpu(), preds.cpu(), average='weighted')
    recall = recall_score(labels.cpu(), preds.cpu(), average='weighted')
    auroc = roc_auc_score(labels.cpu(), torch.softmax(aggregated_outputs, dim=1).cpu(), multi_class='ovr') if aggregation_method == "averaging_vectors" else None

    return f1, accuracy, precision, recall, auroc

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

S = [5, 10, 20] 
R = [5, 10, 20]  
epochs = [2, 4, 6] 

train_dataset, test_dataset = load_data()
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
results = {}
for s in S:
    for r in R:
        print(f"Training with S={s}, R={r}")
        trained_models = sisa_training(train_dataset, s, r, epochs)

        print(f"Evaluating with Majority Vote for S={s}, R={r}")
        metrics_majority_vote = evaluate_model(trained_models, test_loader, aggregation_method="majority_vote")

        print(f"Evaluating with Averaging Prediction Vectors for S={s}, R={r}")
        metrics_averaging_vectors = evaluate_model(trained_models, test_loader, aggregation_method="averaging_vectors")

        results[(s, r)] = {
            "majority_vote": {
                "F1-Score": metrics_majority_vote[0],
                "Accuracy": metrics_majority_vote[1],
                "Precision": metrics_majority_vote[2],
                "Recall": metrics_majority_vote[3],
                "AUROC": metrics_majority_vote[4]
            },
            "averaging_vectors": {
                "F1-Score": metrics_averaging_vectors[0],
                "Accuracy": metrics_averaging_vectors[1],
                "Precision": metrics_averaging_vectors[2],
                "Recall": metrics_averaging_vectors[3],
                "AUROC": metrics_averaging_vectors[4]
            }
        }

for key, value in results.items():
    s, r = key
    print(f"Results for S={s}, R={r}")
    print("Majority Vote:")
    print(f"  F1-Score: {value['majority_vote']['F1-Score']}")
    print(f"  Accuracy: {value['majority_vote']['Accuracy']}")
    print(f"  Precision: {value['majority_vote']['Precision']}")
    print(f"  Recall: {value['majority_vote']['Recall']}")
    print(f"  AUROC: {value['majority_vote']['AUROC']}")

    print("Averaging Prediction Vectors:")
    print(f"  F1-Score: {value['averaging_vectors']['F1-Score']}")
    print(f"  Accuracy: {value['averaging_vectors']['Accuracy']}")
    print(f"  Precision: {value['averaging_vectors']['Precision']}")
    print(f"  Recall: {value['averaging_vectors']['Recall']}")
    print(f"  AUROC: {value['averaging_vectors']['AUROC']}")
    print()

Training with S=5, R=5


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 158MB/s]
Training for 4 epochs: 100%|██████████| 4/4 [00:15<00:00,  3.77s/it]


Evaluating with Majority Vote for S=5, R=5
Evaluating with Averaging Prediction Vectors for S=5, R=5
Training with S=5, R=10


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Training for 2 epochs: 100%|██████████| 2/2 [00:03<00:00,  1.79s/it]


Evaluating with Majority Vote for S=5, R=10
Evaluating with Averaging Prediction Vectors for S=5, R=10
Training with S=5, R=20


...

Evaluating with Majority Vote for S=20, R=5
Evaluating with Averaging Prediction Vectors for S=20, R=5
Training with S=20, R=10


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Training for 2 epochs: 100%|██████████| 2/2 [00:00<00:00,  2.26it/s]


Evaluating with Majority Vote for S=20, R=10
Evaluating with Averaging Prediction Vectors for S=20, R=10
Training with S=20, R=20


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Training for 4 epochs: 100%|██████████| 4/4 [00:00<00:00,  4.52it/s]


Evaluating with Majority Vote for S=20, R=20
Evaluating with Averaging Prediction Vectors for S=20, R=20
Results for S=5, R=5
Majority Vote:
  F1-Score: 0.8405181943768092
  Accuracy: 0.8411
  Precision: 0.8420917401839908
  Recall: 0.8411
  AUROC: None
Averaging Prediction Vectors:
  F1-Score: 0.8573073199774002
  Accuracy: 0.8576
  Precision: 0.8594707506318727
  Recall: 0.8576
  AUROC: 0.9876770555555554

Results for S=5, R=10
Majority Vote:
  F1-Score: 0.8197244788103737
  Accuracy: 0.8196
  Precision: 0.8236892528697101
  Recall: 0.8196
  AUROC: None
Averaging Prediction Vectors:
  F1-Score: 0.8317048162725034
  Accuracy: 0.8311
  Precision: 0.8358324201358591
  Recall: 0.8311
  AUROC: 0.9846405888888891

Results for S=5, R=20
Majority Vote:
  F1-Score: 0.8013941611438867
  Accuracy: 0.8001
  Precision: 0.8041845825902085
  Recall: 0.8001
  AUROC: None
Averaging Prediction Vectors:
  F1-Score: 0.8161197218776739
  Accuracy: 0.815
  Precision: 0.8184424606423827
  Recall: 0.815
  A

## 1.2

In [30]:
def remove_data_from_dataset(dataset, indices_to_remove):
    return Subset(dataset, [i for i in range(len(dataset)) if i not in indices_to_remove])

def sisa_unlearning(models, original_dataset, data_to_forget_indices, S, R, epochs_list):
    shard_size = len(original_dataset) // S
    shards = random_split(original_dataset, [shard_size] * S)  # Creating shards
    updated_models = []

    for shard_index, shard in enumerate(shards):
        shard_indices = set(shard.indices)
        forget_indices_in_shard = shard_indices.intersection(data_to_forget_indices)

        if forget_indices_in_shard:
            print(f"Updating shard {shard_index}")
            slice_size = len(shard) // R
            slices = random_split(shard, [slice_size] * R)  # Creating slices
            updated_slices = []
            for slice in slices:
                slice_indices = set(slice.indices)
                updated_slice_indices = list(slice_indices.difference(forget_indices_in_shard))
                updated_slices.append(Subset(original_dataset, updated_slice_indices))

            model = create_model()
            for i, updated_slice in enumerate(updated_slices):
                data_loader = DataLoader(updated_slice, batch_size=32, shuffle=True)
                model = train_shard(model, data_loader, epochs_list[i % len(epochs_list)])
            updated_models.append(model)
        else:
            updated_models.append(models[shard_index])  # No update needed for this shard

    return updated_models

def evaluate_unlearning(original_models, updated_models, data_to_forget_indices, test_loader):
    # Evaluate original models
    print("Evaluating original models")
    original_metrics_majority_vote = evaluate_model(original_models, test_loader, aggregation_method="majority_vote")
    original_metrics_averaging_vectors = evaluate_model(original_models, test_loader, aggregation_method="averaging_vectors")

    # Evaluate updated models
    print("Evaluating updated models")
    updated_metrics_majority_vote = evaluate_model(updated_models, test_loader, aggregation_method="majority_vote")
    updated_metrics_averaging_vectors = evaluate_model(updated_models, test_loader, aggregation_method="averaging_vectors")

    # Ensure forget indices are within the range of the test dataset
    data_to_forget_indices = [idx for idx in data_to_forget_indices if idx < len(test_loader.dataset)]

    # Evaluate forgetting effectiveness
    forget_data_loader = DataLoader(Subset(test_loader.dataset, data_to_forget_indices), batch_size=32, shuffle=False)
    print("Evaluating forgetting effectiveness on original models")
    original_forget_preds = aggregate_majority_vote(original_models, forget_data_loader)
    print("Evaluating forgetting effectiveness on updated models")
    updated_forget_preds = aggregate_majority_vote(updated_models, forget_data_loader)

    return {
        "original_majority_vote": original_metrics_majority_vote,
        "original_averaging_vectors": original_metrics_averaging_vectors,
        "updated_majority_vote": updated_metrics_majority_vote,
        "updated_averaging_vectors": updated_metrics_averaging_vectors,
        "original_forget_preds": original_forget_preds,
        "updated_forget_preds": updated_forget_preds,
    }

In [31]:
forget_indices = np.random.choice(len(train_dataset), 500, replace=False)
trained_models = sisa_training(train_dataset, S[0], R[0], epochs)
updated_models = sisa_unlearning(trained_models, train_dataset, forget_indices, S[0], R[0], epochs)
unlearning_results = evaluate_unlearning(trained_models, updated_models, forget_indices, test_loader)

print("Original Metrics (Majority Vote): "+"f1 : "+str(unlearning_results["original_majority_vote"][0])+" accuracy : "+str(unlearning_results["original_majority_vote"][1])+" precision : "+str(unlearning_results["original_majority_vote"][2])
     +" recall : "+str(unlearning_results["original_majority_vote"][3])+" auroc : "+str(unlearning_results["original_majority_vote"][4]))
print("Updated Metrics (Majority Vote): "+"f1 : "+str(unlearning_results["updated_majority_vote"][0])+" accuracy : "+str(unlearning_results["updated_majority_vote"][1])+" precision : "+str(unlearning_results["updated_majority_vote"][2])
     +" recall : "+str(unlearning_results["updated_majority_vote"][3])+" auroc : "+str(unlearning_results["updated_majority_vote"][4]))
print("Original Metrics (Averaging Vectors): "+"f1 : "+str(unlearning_results["original_averaging_vectors"][0])+" accuracy : "+str(unlearning_results["original_averaging_vectors"][1])+" precision : "+str(unlearning_results["original_averaging_vectors"][2])
     +" recall : "+str(unlearning_results["original_averaging_vectors"][3])+" auroc : "+str(unlearning_results["original_averaging_vectors"][4]))
print("Updated Metrics (Averaging Vectors): "+"f1 : "+str(unlearning_results["updated_averaging_vectors"][0])+" accuracy : "+str(unlearning_results["updated_averaging_vectors"][1])+" precision : "+str(unlearning_results["updated_averaging_vectors"][2])
     +" recall : "+str(unlearning_results["updated_averaging_vectors"][3])+" auroc : "+str(unlearning_results["updated_averaging_vectors"][4]))


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Training for 4 epochs: 100%|██████████| 4/4 [00:14<00:00,  3.51s/it]


Updating shard 0


Training for 4 epochs: 100%|██████████| 4/4 [00:14<00:00,  3.51s/it]


Updating shard 1


Training for 4 epochs: 100%|██████████| 4/4 [00:14<00:00,  3.51s/it]


...

Updating shard 3


Training for 4 epochs: 100%|██████████| 4/4 [00:14<00:00,  3.51s/it]


Updating shard 4


Training for 4 epochs: 100%|██████████| 4/4 [00:14<00:00,  3.52s/it]


Evaluating original models
Evaluating updated models
Evaluating forgetting effectiveness on original models
Evaluating forgetting effectiveness on updated models
Original Metrics (Majority Vote): f1 : 0.8398683603665354 accuracy : 0.8391 precision : 0.8416427910029636 recall : 0.8391 auroc : None
Updated Metrics (Majority Vote): f1 : 0.8294084431399037 accuracy : 0.8274 precision : 0.8352511600968353 recall : 0.8274 auroc : None
Original Metrics (Averaging Vectors): f1 : 0.859401866702944 accuracy : 0.8593 precision : 0.8611902935670638 recall : 0.8593 auroc : 0.9876866444444443
Updated Metrics (Averaging Vectors): f1 : 0.8389655619952291 accuracy : 0.8371 precision : 0.8448097211672988 recall : 0.8371 auroc : 0.9848228666666665


## 1.3

In [10]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
import torchvision.transforms as transforms

In [11]:
def calculate_losses_with_augmentation(models, data_loader, device, augment=False):
    criterion = torch.nn.CrossEntropyLoss(reduction='none').to(device)
    losses = []
    aug_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    ])

    with torch.no_grad():
        for inputs, labels in data_loader:
            if augment:
                inputs = aug_transform(inputs)
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = [model(inputs) for model in models]
            avg_outputs = torch.mean(torch.stack(outputs), dim=0)
            loss = criterion(avg_outputs, labels)
            losses.extend(loss.cpu().numpy())

    return np.array(losses)

def prepare_dataset(losses1, losses2):
    X = np.concatenate((losses1, losses2)).reshape(-1, 1)
    y = np.concatenate((np.ones(len(losses1)), np.zeros(len(losses2))))
    return X, y

def membership_inference_attack_cv(losses1, losses2):
    X, y = prepare_dataset(losses1, losses2)
    param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
    clf = GridSearchCV(LogisticRegression(solver='liblinear'), param_grid, cv=5, scoring='accuracy')
    clf.fit(X, y)
    return clf.best_score_




In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

forget_loader = DataLoader(Subset(train_dataset, forget_indices), batch_size=32, shuffle=False)
test_indices = np.random.choice(len(test_loader.dataset), 500, replace=False)
test_loader_subset = DataLoader(Subset(test_loader.dataset, test_indices), batch_size=32, shuffle=False)

trained_forget_losses = calculate_losses_with_augmentation(trained_models, forget_loader, device, augment=True)
trained_test_losses = calculate_losses_with_augmentation(trained_models, test_loader_subset, device, augment=True)

unlearned_forget_losses = calculate_losses_with_augmentation(updated_models, forget_loader, device, augment=True)
unlearned_test_losses = calculate_losses_with_augmentation(updated_models, test_loader_subset, device, augment=True)

trained_score_1_2 = membership_inference_attack_cv(trained_forget_losses, trained_test_losses)
unlearned_score_3_4 = membership_inference_attack_cv(unlearned_forget_losses, unlearned_test_losses)
trained_score_1_2_unlearned = membership_inference_attack_cv(trained_forget_losses, unlearned_forget_losses)
trained_score_3_4_unlearned = membership_inference_attack_cv(trained_test_losses, unlearned_test_losses)

# Print results
print(f"Cross-validation of datasets 1 and 2 for the trained model: {trained_score_1_2 * 100:.2f}%")
print(f"Cross-validation of datasets 3 and 4 for the unlearned model: {unlearned_score_3_4 * 100:.2f}%")
print(f"Cross-validation of datasets 1 and 2 for the trained and unlearned model: {trained_score_1_2_unlearned * 100:.2f}%")
print(f"Cross-validation of datasets 3 and 4 for the trained and unlearned model: {trained_score_3_4_unlearned * 100:.2f}%")

Cross-validation of datasets 1 and 2 for the trained model: 53.10%
Cross-validation of datasets 3 and 4 for the unlearned model: 49.20%
Cross-validation of datasets 1 and 2 for the trained and unlearned model: 50.80%
Cross-validation of datasets 3 and 4 for the trained and unlearned model: 50.30%


# 1.4

In [32]:
import torch
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split, Subset
from torch.multiprocessing import set_start_method
from tqdm import tqdm

In [33]:
try:
    set_start_method('spawn')
except RuntimeError:
    pass

def load_data():
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    return train_set, test_set

train_dataset, test_dataset = load_data()
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def add_backdoor_trigger(data, trigger_size=3):
    data = data.clone()
    if len(data.shape) == 3:  # If data is a single image
        _, height, width = data.shape
        start_x = np.random.randint(0, height - trigger_size)
        start_y = np.random.randint(0, width - trigger_size)
        data[:, start_x:start_x + trigger_size, start_y:start_y + trigger_size] = 0  # Add black block
    elif len(data.shape) == 4:  # If data is a batch of images
        batch_size, _, height, width = data.shape
        start_x = np.random.randint(0, height - trigger_size)
        start_y = np.random.randint(0, width - trigger_size)
        for i in range(batch_size):
            data[i, :, start_x:start_x + trigger_size, start_y:start_y + trigger_size] = 0  # Add black block
    else:
        raise ValueError("Unsupported data shape")
    return data

def create_poisoned_dataset(dataset, class_to_poison, num_poisoned_samples, trigger_size=3):
    print("Creating poisoned dataset...")
    poisoned_indices = [i for i, (_, label) in enumerate(dataset) if label == class_to_poison]
    poisoned_indices = np.random.choice(poisoned_indices, num_poisoned_samples, replace=False)
    
    poisoned_dataset = []
    for i, (data, label) in tqdm(enumerate(dataset), total=len(dataset)):
        if i in poisoned_indices:
            data = add_backdoor_trigger(data, trigger_size)
        poisoned_dataset.append((data, label))
        
    print(f"Poisoned {num_poisoned_samples} samples from class {class_to_poison}.")
    return poisoned_dataset, poisoned_indices

class PoisonedDataset(Dataset):
    def __init__(self, dataset, poisoned_indices, trigger_size=3):
        self.dataset = dataset
        self.poisoned_indices = poisoned_indices
        self.trigger_size = trigger_size

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data, label = self.dataset[idx]
        if idx in self.poisoned_indices:
            data = add_backdoor_trigger(data.unsqueeze(0), self.trigger_size).squeeze(0)
        return data, label

def sisa_training(dataset, S, R, epochs_list):
    shard_size = len(dataset) // S
    shards = random_split(dataset, [shard_size] * S)  # Creating shards
    models = []

    for shard in shards:
        slice_size = len(shard) // R
        slices = random_split(shard, [slice_size] * R)  # Creating slices
        model = create_model()
        for i, slice in enumerate(slices):
            data_loader = DataLoader(slice, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
            model = train_shard(model, data_loader, epochs_list[i % len(epochs_list)])
        models.append(model)
    return models

class SimpleNN(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 128)
        self.fc2 = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def create_model():
    input_size = 32 * 32 * 3  # Adjusted for CIFAR-10 images (32x32x3)
    num_classes = 10  # 10 classes for CIFAR-10
    model = SimpleNN(input_size, num_classes).to(device)
    return model

# Function to train the model on a shard
def train_shard(model, dataloader, epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, labels) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)):
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(dataloader)}")
    
    return model

def evaluate_backdoor_attack(models, test_loader):
    clean_metrics_majority_vote = []
    clean_metrics_averaging_vectors = []
    poisoned_metrics_majority_vote = []
    poisoned_metrics_averaging_vectors = []
    asr_majority_vote = 0
    asr_averaging_vectors = 0

    for model in models:
        model.eval()
    
    # Clean data evaluation
    total_correct_majority_vote = 0
    total_correct_averaging_vectors = 0
    total_samples = 0
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = [model(data) for model in models]
        
        majority_votes = torch.stack(outputs).mean(dim=0).argmax(dim=1)
        total_correct_majority_vote += (majority_votes == labels).sum().item()
        
        averaged_outputs = torch.mean(torch.stack(outputs), dim=0).argmax(dim=1)
        total_correct_averaging_vectors += (averaged_outputs == labels).sum().item()
        
        total_samples += labels.size(0)
    
    clean_accuracy_majority_vote = total_correct_majority_vote / total_samples
    clean_accuracy_averaging_vectors = total_correct_averaging_vectors / total_samples
    clean_metrics_majority_vote.append(clean_accuracy_majority_vote)
    clean_metrics_averaging_vectors.append(clean_accuracy_averaging_vectors)
    
    total_correct_majority_vote = 0
    total_correct_averaging_vectors = 0
    total_samples = 0
    for data, labels in test_loader:
        data = add_backdoor_trigger(data)
        data, labels = data.to(device), labels.to(device)
        outputs = [model(data) for model in models]
        
        majority_votes = torch.stack(outputs).mean(dim=0).argmax(dim=1)
        total_correct_majority_vote += (majority_votes == labels).sum().item()
        
        averaged_outputs = torch.mean(torch.stack(outputs), dim=0).argmax(dim=1)
        total_correct_averaging_vectors += (averaged_outputs == labels).sum().item()
        
        total_samples += labels.size(0)
    
    poisoned_accuracy_majority_vote = total_correct_majority_vote / total_samples
    poisoned_accuracy_averaging_vectors = total_correct_averaging_vectors / total_samples
    poisoned_metrics_majority_vote.append(poisoned_accuracy_majority_vote)
    poisoned_metrics_averaging_vectors.append(poisoned_accuracy_averaging_vectors)
    
    asr_majority_vote = poisoned_accuracy_majority_vote
    asr_averaging_vectors = poisoned_accuracy_averaging_vectors
    
    return {
        "clean_metrics_majority_vote": clean_metrics_majority_vote,
        "clean_metrics_averaging_vectors": clean_metrics_averaging_vectors,
        "poisoned_metrics_majority_vote": poisoned_metrics_majority_vote,
        "poisoned_metrics_averaging_vectors": poisoned_metrics_averaging_vectors,
        "asr_majority_vote": asr_majority_vote,
        "asr_averaging_vectors": asr_averaging_vectors
    }

Files already downloaded and verified
Files already downloaded and verified


In [34]:
S = 5  
R = 5 
epochs = [2, 3, 4, 5, 6] 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

subsample_size = 10000 
train_indices = np.random.choice(len(train_dataset), subsample_size, replace=False)
subsampled_train_dataset = Subset(train_dataset, train_indices)

class_to_poison = 1
num_poisoned_samples = 500
poisoned_train_dataset, poisoned_indices = create_poisoned_dataset(subsampled_train_dataset, class_to_poison, num_poisoned_samples)
poisoned_train_dataset = PoisonedDataset(subsampled_train_dataset, poisoned_indices)

print(f"Training with S={S}, R={R}, epochs={epochs}")
poisoned_models = sisa_training(poisoned_train_dataset, S, R, epochs)
all_models = poisoned_models

test_subsample_size = 2000 
test_indices = np.random.choice(len(test_dataset), test_subsample_size, replace=False)
subsampled_test_dataset = Subset(test_dataset, test_indices)

print("Evaluating model performance on backdoor attack...")
test_loader = DataLoader(subsampled_test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
backdoor_results = evaluate_backdoor_attack(all_models, test_loader)

print("Backdoor Attack Results (Poisoned Model):")
print("Poisoned Metrics (Majority Vote):", backdoor_results["poisoned_metrics_majority_vote"])
print("Poisoned Metrics (Averaging Vectors):", backdoor_results["poisoned_metrics_averaging_vectors"])
print("Attack Success Rate (Majority Vote):", backdoor_results["asr_majority_vote"])
print("Attack Success Rate (Averaging Vectors):", backdoor_results["asr_averaging_vectors"])

Creating poisoned dataset...


100%|██████████| 10000/10000 [00:03<00:00, 3280.44it/s]


Poisoned 500 samples from class 1.
Training with S=5, R=5, epochs=[2, 3, 4, 5, 6]


Epoch 1/2, Loss: 2.2220503183511586


Epoch 2/2, Loss: 2.0144766660837026


...

Epoch 2/6, Loss: 1.7110718947190504


Epoch 3/6, Loss: 1.6177898462002094


Epoch 4/6, Loss: 1.5098895659813514


Epoch 5/6, Loss: 1.4227650990852942


Epoch 6/6, Loss: 1.3525173572393565
Evaluating model performance on backdoor attack...
Backdoor Attack Results (Poisoned Model):
Poisoned Metrics (Majority Vote): [0.3775]
Poisoned Metrics (Averaging Vectors): [0.3775]
Attack Success Rate (Majority Vote): 0.3775
Attack Success Rate (Averaging Vectors): 0.3775


In [35]:
import torch
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split, Subset
from torch.multiprocessing import set_start_method
from tqdm import tqdm

In [36]:
try:
    set_start_method('spawn')
except RuntimeError:
    pass

def load_data():
    transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    return train_set, test_set

train_dataset, test_dataset = load_data()
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class PoisonedDataset(Dataset):
    def __init__(self, dataset, poisoned_indices, trigger_size=3):
        self.dataset = dataset
        self.poisoned_indices = poisoned_indices
        self.trigger_size = trigger_size

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data, label = self.dataset[idx]
        if idx in self.poisoned_indices:
            data = add_backdoor_trigger(data.unsqueeze(0), self.trigger_size).squeeze(0)
        return data, label

class SimpleNN(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 128)
        self.fc2 = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def create_model():
    input_size = 32 * 32 * 3  # Adjusted for CIFAR-10 images (32x32x3)
    num_classes = 10  # 10 classes for CIFAR-10
    model = SimpleNN(input_size, num_classes).to(device)
    return model

def train_shard(model, dataloader, epochs):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, labels) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)):
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(dataloader)}")
    
    return model

def sisa_training(dataset, S, R, epochs_list):
    shard_size = len(dataset) // S
    shards = random_split(dataset, [shard_size] * S)  # Creating shards
    models = []

    for shard in shards:
        slice_size = len(shard) // R
        slices = random_split(shard, [slice_size] * R)  # Creating slices
        model = create_model()
        for i, slice in enumerate(slices):
            data_loader = DataLoader(slice, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
            model = train_shard(model, data_loader, epochs_list[i % len(epochs_list)])
        models.append(model)
    return models

def add_backdoor_trigger(data, trigger_size=3):
    data = data.clone()
    if len(data.shape) == 3:  # If data is a single image
        _, height, width = data.shape
        start_x = np.random.randint(0, height - trigger_size)
        start_y = np.random.randint(0, width - trigger_size)
        data[:, start_x:start_x + trigger_size, start_y:start_y + trigger_size] = 0  # Add black block
    elif len(data.shape) == 4:  # If data is a batch of images
        batch_size, _, height, width = data.shape
        start_x = np.random.randint(0, height - trigger_size)
        start_y = np.random.randint(0, width - trigger_size)
        for i in range(batch_size):
            data[i, :, start_x:start_x + trigger_size, start_y:start_y + trigger_size] = 0  # Add black block
    else:
        raise ValueError("Unsupported data shape")
    return data

def evaluate_backdoor_attack(models, test_loader):
    clean_metrics_majority_vote = []
    clean_metrics_averaging_vectors = []
    poisoned_metrics_majority_vote = []
    poisoned_metrics_averaging_vectors = []
    asr_majority_vote = 0
    asr_averaging_vectors = 0

    for model in models:
        model.eval()
    
    total_correct_majority_vote = 0
    total_correct_averaging_vectors = 0
    total_samples = 0
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = [model(data) for model in models]
        
        majority_votes = torch.stack(outputs).mean(dim=0).argmax(dim=1)
        total_correct_majority_vote += (majority_votes == labels).sum().item()
        
        averaged_outputs = torch.mean(torch.stack(outputs), dim=0).argmax(dim=1)
        total_correct_averaging_vectors += (averaged_outputs == labels).sum().item()
        
        total_samples += labels.size(0)
    
    clean_accuracy_majority_vote = total_correct_majority_vote / total_samples
    clean_accuracy_averaging_vectors = total_correct_averaging_vectors / total_samples
    clean_metrics_majority_vote.append(clean_accuracy_majority_vote)
    clean_metrics_averaging_vectors.append(clean_accuracy_averaging_vectors)
    
    total_correct_majority_vote = 0
    total_correct_averaging_vectors = 0
    total_samples = 0
    for data, labels in test_loader:
        data = add_backdoor_trigger(data)
        data, labels = data.to(device), labels.to(device)
        outputs = [model(data) for model in models]
        
        majority_votes = torch.stack(outputs).mean(dim=0).argmax(dim=1)
        total_correct_majority_vote += (majority_votes == labels).sum().item()
        
        averaged_outputs = torch.mean(torch.stack(outputs), dim=0).argmax(dim=1)
        total_correct_averaging_vectors += (averaged_outputs == labels).sum().item()
        
        total_samples += labels.size(0)
    
    poisoned_accuracy_majority_vote = total_correct_majority_vote / total_samples
    poisoned_accuracy_averaging_vectors = total_correct_averaging_vectors / total_samples
    poisoned_metrics_majority_vote.append(poisoned_accuracy_majority_vote)
    poisoned_metrics_averaging_vectors.append(poisoned_accuracy_averaging_vectors)
    
    asr_majority_vote = poisoned_accuracy_majority_vote
    asr_averaging_vectors = poisoned_accuracy_averaging_vectors
    
    return {
        "clean_metrics_majority_vote": clean_metrics_majority_vote,
        "clean_metrics_averaging_vectors": clean_metrics_averaging_vectors,
        "poisoned_metrics_majority_vote": poisoned_metrics_majority_vote,
        "poisoned_metrics_averaging_vectors": poisoned_metrics_averaging_vectors,
        "asr_majority_vote": asr_majority_vote,
        "asr_averaging_vectors": asr_averaging_vectors
    }

def unlearn_data(models, unlearn_indices, train_dataset, S, R, epochs_list):
    unlearned_models = []
    for model in models:
        new_model = create_model()
        
        remaining_indices = list(set(range(len(train_dataset))) - set(unlearn_indices))
        remaining_dataset = Subset(train_dataset, remaining_indices)
        
        unlearned_model = sisa_training(remaining_dataset, S, R, epochs_list)
        unlearned_models.append(unlearned_model)
        
    return unlearned_models

Files already downloaded and verified
Files already downloaded and verified


In [37]:
S = 5  
R = 5
epochs = [2, 3, 4, 5, 6]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

subsample_size = 10000
train_indices = np.random.choice(len(train_dataset), subsample_size, replace=False)
subsampled_train_dataset = Subset(train_dataset, train_indices)

print("Unlearning the poisoned data and retraining the models...")
unlearned_models = unlearn_data(poisoned_models, poisoned_indices, subsampled_train_dataset, S, R, epochs)

flattened_unlearned_models = [model for shard in unlearned_models for model in shard]

print("Evaluating model performance after unlearning backdoor attack...")
unlearned_backdoor_results = evaluate_backdoor_attack(flattened_unlearned_models, test_loader)

print("Backdoor Attack Results (Unlearned Model):")
print("Clean Metrics (Majority Vote):", unlearned_backdoor_results["clean_metrics_majority_vote"])
print("Clean Metrics (Averaging Vectors):", unlearned_backdoor_results["clean_metrics_averaging_vectors"])
print("Poisoned Metrics (Majority Vote):", unlearned_backdoor_results["poisoned_metrics_majority_vote"])
print("Poisoned Metrics (Averaging Vectors):", unlearned_backdoor_results["poisoned_metrics_averaging_vectors"])
print("Attack Success Rate (Majority Vote):", unlearned_backdoor_results["asr_majority_vote"])
print("Attack Success Rate (Averaging Vectors):", unlearned_backdoor_results["asr_averaging_vectors"])


Unlearning the poisoned data and retraining the models...


Epoch 1/2, Loss: 2.257439653078715


Epoch 2/2, Loss: 2.047386815150579


Epoch 1/3, Loss: 2.088805536429087


Epoch 2/3, Loss: 1.928194522857666


...

Epoch 2/6, Loss: 1.6867198745409648


Epoch 3/6, Loss: 1.575289636850357


Epoch 4/6, Loss: 1.4765307505925496


Epoch 5/6, Loss: 1.3932240704695384


Epoch 6/6, Loss: 1.3254071176052094
Evaluating model performance after unlearning backdoor attack...
Backdoor Attack Results (Unlearned Model):
Clean Metrics (Majority Vote): [0.6937]
Clean Metrics (Averaging Vectors): [0.6937]
Poisoned Metrics (Majority Vote): [0.2364]
Poisoned Metrics (Averaging Vectors): [0.2364]
Attack Success Rate (Majority Vote): 0.2364
Attack Success Rate (Averaging Vectors): 0.2364
